In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import os

#  Import and read the charity_data.csv.
application_df = pd.read_csv("./resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# PRE-PROCESSING

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'],1)

In [3]:
# Look at INCOME_AMT value counts for binning
i_amts = application_df.INCOME_AMT.value_counts()
i_amts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

### Leaving Income amounts as there is only 9 bins here 

In [4]:
# Look at ASK_AMT value counts for binning
a_amts = application_df.ASK_AMT.value_counts()
a_amts

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [5]:
#Bin ask amount values in increaments of 50000

replace_ask = list(a_amts[a_amts<1000].index)

#Binning in dataframe
for amt in replace_ask:
    if amt<=49999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('5000-49999'))
    elif amt<=99999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('50000-99999'))
    elif amt<=499999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('100000-499999'))
    elif amt<=999999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('500000-999999'))
    elif amt<=4999999:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('1M-5M'))
    else:
        application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,str('5M+'))

application_df.ASK_AMT = application_df.ASK_AMT.astype(str)

#checking the binning
application_df.ASK_AMT.value_counts()

5000             25398
5000-49999        2947
100000-499999     2305
50000-99999       1421
1M-5M              940
500000-999999      651
5M+                637
Name: ASK_AMT, dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts<500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(class_count[class_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [7]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(application_cat, axis=1)
application_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_100000-499999,ASK_AMT_1M-5M,ASK_AMT_5000,ASK_AMT_5000-49999,ASK_AMT_50000-99999,ASK_AMT_500000-999999,ASK_AMT_5M+
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34296,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34297,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop('IS_SUCCESSFUL', 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=25, stratify=y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#adding more neurons for analyzing
num_input = len(X_train_scaled[0]) 
h_layer_1 = 100
h_layer_2 = 40

nn = tf.keras.models.Sequential()
asd = tf.keras.layers.Dense

# First hidden layer
nn.add(asd(input_dim=num_input, units=h_layer_1, activation='relu'))

# Second hidden layer
nn.add(asd(units=h_layer_2, activation='relu'))

# Output layer
nn.add(asd(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               5000      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                4040      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 41        
Total params: 9,081
Trainable params: 9,081
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [12]:
#Create a callback that saves the model's weights every 5 epochs.

os.makedirs('./checkpoints/checkpoints_added_neuron/', exist_ok=True)
checkpoint_path='./checkpoints/checkpoints_added_neuron/weights.{epoch:02d}.hdf5'

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only = True,
    period = 5
    )

In [13]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=150, callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 565us/step - loss: 0.5817 - accuracy: 0.7099
Epoch 2/150
804/804 [==============================] - 0s 579us/step - loss: 0.5532 - accuracy: 0.7263
Epoch 3/150
804/804 [==============================] - 0s 577us/step - loss: 0.5495 - accuracy: 0.7289
Epoch 4/150
804/804 [==============================] - 0s 565us/step - loss: 0.5496 - accuracy: 0.7325
Epoch 5/150
804/804 [==============================] - 0s 575us/step - loss: 0.5450 - accuracy: 0.7300

Epoch 00005: saving model to ./checkpoints/checkpoints_added_neuron\weights.05.hdf5
Epoch 6/150
804/804 [==============================] - 0s 572us/step - loss: 0.5461 - accuracy: 0.7320
Epoch 7/150
804/804 [==============================] - 0s 578us/step - loss: 0.5393 - accuracy: 0.7373
Epoch 8/150
804/804 [==============================] - 0s 572us/step - loss: 0.5454 - accuracy: 0.7338
Epoch 9/150
804/804 [==============================] - 0s 579us/step - loss: 0.5450 - accur

804/804 [==============================] - 0s 577us/step - loss: 0.5264 - accuracy: 0.7427
Epoch 70/150
804/804 [==============================] - 0s 597us/step - loss: 0.5215 - accuracy: 0.7459

Epoch 00070: saving model to ./checkpoints/checkpoints_added_neuron\weights.70.hdf5
Epoch 71/150
804/804 [==============================] - 0s 567us/step - loss: 0.5262 - accuracy: 0.7443
Epoch 72/150
804/804 [==============================] - 0s 575us/step - loss: 0.5188 - accuracy: 0.7487
Epoch 73/150
804/804 [==============================] - 0s 565us/step - loss: 0.5220 - accuracy: 0.7429
Epoch 74/150
804/804 [==============================] - 0s 573us/step - loss: 0.5195 - accuracy: 0.7462
Epoch 75/150
804/804 [==============================] - 0s 565us/step - loss: 0.5238 - accuracy: 0.7434

Epoch 00075: saving model to ./checkpoints/checkpoints_added_neuron\weights.75.hdf5
Epoch 76/150
804/804 [==============================] - 0s 573us/step - loss: 0.5242 - accuracy: 0.7455
Epoch 77/15

804/804 [==============================] - 0s 573us/step - loss: 0.5176 - accuracy: 0.7459
Epoch 137/150
804/804 [==============================] - 0s 573us/step - loss: 0.5211 - accuracy: 0.7450
Epoch 138/150
804/804 [==============================] - 0s 569us/step - loss: 0.5202 - accuracy: 0.7444
Epoch 139/150
804/804 [==============================] - 0s 569us/step - loss: 0.5214 - accuracy: 0.7440
Epoch 140/150
804/804 [==============================] - 0s 568us/step - loss: 0.5245 - accuracy: 0.7399

Epoch 00140: saving model to ./checkpoints/checkpoints_added_neuron\weights.140.hdf5
Epoch 141/150
804/804 [==============================] - 0s 572us/step - loss: 0.5217 - accuracy: 0.7460
Epoch 142/150
804/804 [==============================] - 0s 594us/step - loss: 0.5189 - accuracy: 0.7478
Epoch 143/150
804/804 [==============================] - 0s 574us/step - loss: 0.5202 - accuracy: 0.7478
Epoch 144/150
804/804 [==============================] - 0s 570us/step - loss: 0.5191 - 

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7902 - accuracy: 0.7270
Loss: 0.7901535630226135, Accuracy: 0.7269970774650574


In [15]:
#adding more layers for analyzing
num_input = len(X_train_scaled[0]) 
h_layer_1 = 100
h_layer_2 = 35
h_layer_3 = 10

nn_new = tf.keras.models.Sequential()
asd = tf.keras.layers.Dense

# First hidden layer
nn_new.add(asd(input_dim=num_input, units=h_layer_1, activation='relu'))

# Second hidden layer
nn_new.add(asd(units=h_layer_2, activation='relu'))

# Third hidden layer
nn_new.add(asd(units=h_layer_3, activation='relu'))

# Output layer
nn_new.add(asd(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn_new.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               5000      
_________________________________________________________________
dense_4 (Dense)              (None, 35)                3535      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                360       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 8,906
Trainable params: 8,906
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [17]:
#Create a callback that saves the model's weights every 5 epochs.

os.makedirs('./checkpoints/checkpoints_added_layer/', exist_ok=True)
checkpoint_path='./checkpoints/checkpoints_added_layer/weights.{epoch:02d}.hdf5'

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only = True,
    period = 5
    )

# Train the model
nn_new.fit(X_train_scaled, y_train, epochs=150, callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 605us/step - loss: 0.5995 - accuracy: 0.6986
Epoch 2/150
804/804 [==============================] - 0s 615us/step - loss: 0.5466 - accuracy: 0.7352
Epoch 3/150
804/804 [==============================] - 0s 606us/step - loss: 0.5472 - accuracy: 0.7352
Epoch 4/150
804/804 [==============================] - 0s 611us/step - loss: 0.5418 - accuracy: 0.7379
Epoch 5/150
804/804 [==============================] - 0s 601us/step - loss: 0.5424 - accuracy: 0.7378

Epoch 00005: saving model to ./checkpoints/checkpoints_added_layer\weights.05.hdf5
Epoch 6/150
804/804 [==============================] - 0s 603us/step - loss: 0.5429 - accuracy: 0.7337
Epoch 7/150
804/804 [==============================] - 0s 599us/step - loss: 0.5384 - accuracy: 0.7341
Epoch 8/150
804/804 [==============================] - 0s 599us/step - loss: 0.5401 - accuracy: 0.7367
Epoch 9/150
804/804 [==============================] - 0s 599us/step - loss: 0.5391 - accura

804/804 [==============================] - 1s 645us/step - loss: 0.5250 - accuracy: 0.7414
Epoch 69/150
804/804 [==============================] - 0s 618us/step - loss: 0.5223 - accuracy: 0.7434
Epoch 70/150
804/804 [==============================] - 1s 667us/step - loss: 0.5276 - accuracy: 0.7394

Epoch 00070: saving model to ./checkpoints/checkpoints_added_layer\weights.70.hdf5
Epoch 71/150
804/804 [==============================] - 1s 634us/step - loss: 0.5239 - accuracy: 0.7435
Epoch 72/150
804/804 [==============================] - 0s 618us/step - loss: 0.5202 - accuracy: 0.7448
Epoch 73/150
804/804 [==============================] - 1s 626us/step - loss: 0.5209 - accuracy: 0.7446
Epoch 74/150
804/804 [==============================] - 1s 700us/step - loss: 0.5238 - accuracy: 0.7429
Epoch 75/150
804/804 [==============================] - 1s 685us/step - loss: 0.5222 - accuracy: 0.7447

Epoch 00075: saving model to ./checkpoints/checkpoints_added_layer\weights.75.hdf5
Epoch 76/150



Epoch 00135: saving model to ./checkpoints/checkpoints_added_layer\weights.135.hdf5
Epoch 136/150
804/804 [==============================] - 0s 600us/step - loss: 0.5204 - accuracy: 0.7482
Epoch 137/150
804/804 [==============================] - 0s 599us/step - loss: 0.5180 - accuracy: 0.7476
Epoch 138/150
804/804 [==============================] - 0s 597us/step - loss: 0.5206 - accuracy: 0.74470s - loss: 0.5211 - ac
Epoch 139/150
804/804 [==============================] - 0s 601us/step - loss: 0.5212 - accuracy: 0.7442
Epoch 140/150
804/804 [==============================] - 0s 593us/step - loss: 0.5199 - accuracy: 0.7483

Epoch 00140: saving model to ./checkpoints/checkpoints_added_layer\weights.140.hdf5
Epoch 141/150
804/804 [==============================] - 0s 597us/step - loss: 0.5228 - accuracy: 0.7412
Epoch 142/150
804/804 [==============================] - 0s 595us/step - loss: 0.5187 - accuracy: 0.7444
Epoch 143/150
804/804 [==============================] - 0s 598us/step - 

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8209 - accuracy: 0.7282
Loss: 0.8209224939346313, Accuracy: 0.7281632423400879


## testing with different activation functions

In [19]:
#using different activation for analyzing
num_input = len(X_train_scaled[0]) 
h_layer_1 = 100
h_layer_2 = 45
h_layer_3 = 10

nn_new_two = tf.keras.models.Sequential()
asd = tf.keras.layers.Dense

# First hidden layer
nn_new_two.add(asd(input_dim=num_input, units=h_layer_1, activation='tanh'))

# Second hidden layer
nn_new_two.add(asd(units=h_layer_2, activation='tanh'))

# Third hidden layer
nn_new_two.add(asd(units=h_layer_3, activation='tanh'))

# Output layer
nn_new_two.add(asd(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn_new_two.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               5000      
_________________________________________________________________
dense_8 (Dense)              (None, 45)                4545      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                460       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 10,016
Trainable params: 10,016
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn_new_two.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [21]:
#Create a callback that saves the model's weights every 5 epochs.

os.makedirs('./checkpoints/checkpoints_different_activation/', exist_ok=True)
checkpoint_path='./checkpoints/checkpoints_different_activation/weights.{epoch:02d}.hdf5'

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only = True,
    period = 5
    )

# Train the model
nn_new_two.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 636us/step - loss: 0.5853 - accuracy: 0.7051
Epoch 2/100
804/804 [==============================] - 1s 639us/step - loss: 0.5604 - accuracy: 0.7236
Epoch 3/100
804/804 [==============================] - 1s 633us/step - loss: 0.5519 - accuracy: 0.7277
Epoch 4/100
804/804 [==============================] - 1s 640us/step - loss: 0.5474 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 1s 643us/step - loss: 0.5441 - accuracy: 0.7341

Epoch 00005: saving model to ./checkpoints/checkpoints_different_activation\weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 1s 640us/step - loss: 0.5439 - accuracy: 0.7351
Epoch 7/100
804/804 [==============================] - 1s 651us/step - loss: 0.5440 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 1s 733us/step - loss: 0.5406 - accuracy: 0.7370
Epoch 9/100
804/804 [==============================] - 1s 664us/step - loss: 0.5394

804/804 [==============================] - 0s 601us/step - loss: 0.5176 - accuracy: 0.7507
Epoch 67/100
804/804 [==============================] - 0s 599us/step - loss: 0.5205 - accuracy: 0.7435
Epoch 68/100
804/804 [==============================] - 0s 608us/step - loss: 0.5201 - accuracy: 0.7479
Epoch 69/100
804/804 [==============================] - 0s 601us/step - loss: 0.5192 - accuracy: 0.7450
Epoch 70/100
804/804 [==============================] - 0s 605us/step - loss: 0.5177 - accuracy: 0.7464

Epoch 00070: saving model to ./checkpoints/checkpoints_different_activation\weights.70.hdf5
Epoch 71/100
804/804 [==============================] - 0s 601us/step - loss: 0.5172 - accuracy: 0.7467
Epoch 72/100
804/804 [==============================] - 1s 630us/step - loss: 0.5208 - accuracy: 0.7460
Epoch 73/100
804/804 [==============================] - 0s 606us/step - loss: 0.5205 - accuracy: 0.7435
Epoch 74/100
804/804 [==============================] - 0s 605us/step - loss: 0.5180 - a

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new_two.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5763 - accuracy: 0.7285
Loss: 0.5762902498245239, Accuracy: 0.7285131216049194


In [23]:
#Finally, Save the model
nn_new_two.save("./AlphabetSoupCharity_Optimization.h5")